In [14]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Directories for train and test data
train_data_dir = '/content/drive/MyDrive/gantrainfile/train'
test_data_dir = '/content/drive/MyDrive/gantestfile/'

# Image size for the model
IMAGE_SIZE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 15

# Class labels mapping corresponding to folder names in the training directory
class_labels = {0: 'basketball_court', 1: 'bridge', 2: 'crosswalk', 3: 'golfcourse',
                4: 'oil_well', 5: 'overpass', 6: 'railway', 7: 'runway',
                8: 'swimming_pool', 9: 'tennis_court'}

# Data Augmentation for training images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Only rescale test images
test_datagen = ImageDataGenerator(rescale=1./255)

# Load training data from subdirectories (each subdirectory corresponds to a class)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

# Load test data (all images in the 'test' folder, no subdirectories)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=IMAGE_SIZE,
    batch_size=1,  # Batch size is 1 to preserve the order of predictions
    class_mode=None,
    shuffle=False
)

# Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(10, activation='softmax')  # 10 classes for output
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model without validation data
history = model.fit(
    train_generator,
    epochs=EPOCHS
)

# Predictions on test set
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Generate CSV submission file
image_ids = [int(os.path.basename(path).split('.')[0]) for path in test_generator.filenames]
submission = pd.DataFrame({'ImageID': image_ids, 'Label': predicted_classes})
submission.to_csv('yourrollnumber.csv', index=False)


Found 500 images belonging to 10 classes.
Found 100 images belonging to 1 classes.


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_23 (Conv2D)                   │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_23 (MaxPooling2D)      │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_24 (Conv2D)                   │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_24 (MaxPooling2D)      │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_25 (Conv2D)                   │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_25 (MaxPooling2D)      │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_26 (Conv2D)                   │ (None, 15, 15, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_26 (MaxPooling2D)      │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 512)                 │       3,211,776 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,457,738 (13.19 MB)

 Trainable params: 3,457,738 (13.19 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 303ms/step - accuracy: 0.1015 - loss: 2.3230
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 212ms/step - accuracy: 0.1820 - loss: 2.0916
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 159ms/step - accuracy: 0.2714 - loss: 1.9660
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 158ms/step - accuracy: 0.3701 - loss: 1.6180
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 223ms/step - accuracy: 0.5092 - loss: 1.3185
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 165ms/step - accuracy: 0.5679 - loss: 1.1486
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 164ms/step - accuracy: 0.6359 - loss: 1.0158
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 180ms/step - accuracy: 0.7142 - loss: 0.8323
Epoch 9/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 163ms/step - accuracy: 0.7826 - loss: 0.6688
Epoch 10/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 169ms/step - accuracy: 0.7264 - loss: 0.7836
Epoch 11/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 157ms/step - accuracy: 0.7124 - loss: 0.7480
Epoch 12/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 207ms/ste

In [25]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Directories for train and test data
train_data_dir = '/content/drive/MyDrive/gantrainfile/train'
test_data_dir = '/content/drive/MyDrive/gantestfile/'  # Corrected to point to the 'test' folder

# Image size for the model
IMAGE_SIZE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 15

# Class labels mapping corresponding to folder names in the training directory
class_labels = {0: 'basketball_court', 1: 'bridge', 2: 'crosswalk', 3: 'golfcourse',
                4: 'oil_well', 5: 'overpass', 6: 'railway', 7: 'runway',
                8: 'swimming_pool', 9: 'tennis_court'}

# Data Augmentation for training images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Only rescale test images
test_datagen = ImageDataGenerator(rescale=1./255)

# Load training data from subdirectories (each subdirectory corresponds to a class)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

# Load test data (all images in the 'test' folder, no subdirectories)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=IMAGE_SIZE,
    batch_size=1,  # Batch size is 1 to preserve the order of predictions
    class_mode=None,
    shuffle=False
)

# Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(10, activation='softmax')  # 10 classes for output
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model without validation data
history = model.fit(
    train_generator,
    epochs=EPOCHS
)

# Predictions on test set
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Generate CSV submission file
image_ids = [int(os.path.basename(path).split('.')[0]) for path in test_generator.filenames]
submission = pd.DataFrame({'ImageID': image_ids, 'Label': predicted_classes})
submission.to_csv('test_labels.csv', index=False)

# Optional: Evaluate accuracy on the test dataset if true labels are available
# Assume you have true labels in a CSV file called 'test_labels.csv'
true_labels_df = pd.read_csv('test_labels.csv')  # This should contain two columns: 'ImageID' and 'Label'
true_labels = true_labels_df['Label'].values

# Check if the number of predictions matches the number of true labels
if len(predicted_classes) == len(true_labels):
    # Calculate accuracy
    accuracy = np.mean(predicted_classes == true_labels)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')
else:
    print('The number of predicted classes does not match the number of true labels.')


Found 500 images belonging to 10 classes.
Found 100 images belonging to 1 classes.


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_50 (Conv2D)                   │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_50 (MaxPooling2D)      │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_51 (Conv2D)                   │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_51 (MaxPooling2D)      │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_52 (Conv2D)                   │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_52 (MaxPooling2D)      │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_53 (Conv2D)                   │ (None, 15, 15, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_53 (MaxPooling2D)      │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_13 (Flatten)                 │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 512)                 │       3,211,776 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,457,738 (13.19 MB)

 Trainable params: 3,457,738 (13.19 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 323ms/step - accuracy: 0.1009 - loss: 2.2820
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 154ms/step - accuracy: 0.2495 - loss: 1.9714
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 186ms/step - accuracy: 0.4848 - loss: 1.4677
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 159ms/step - accuracy: 0.5819 - loss: 1.1312
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 159ms/step - accuracy: 0.5939 - loss: 1.1714
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 156ms/step - accuracy: 0.6748 - loss: 0.9140
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 218ms/step - accuracy: 0.7089 - loss: 0.7935
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 163ms/step - accuracy: 0.7598 - loss: 0.6676
Epoch 9/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 161ms/step - accuracy: 0.7153 - loss: 0.8651
Epoch 10/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 189ms/step - accuracy: 0.7847 - loss: 0.6122
Epoch 11/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 163ms/step - accuracy: 0.7824 - loss: 0.5591
Epoch 12/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 163ms/ste